Hello WW2 enthusiasts,

This is a brief overview of the Enigma machine. The Enigma was a cipher device used by the Germans during World War II to encrypt and decrypt messages. It allowed the German military to communicate securely and secretly. Although the Enigma machine was considered unbreakable during the war, the British managed to crack the code, playing a crucial role in the Allied victory.

To better understand how it worked, I recommend watching this video: https://www.youtube.com/watch?v=ybkkiGtJmkM.

In [109]:
import copy
class Enigma:
    def __init__(self, rotor_positions, plugboard_settings=None):
        # Realistic rotor wirings for I, II, III (historically accurate, but not the only possible wirings)
        # they are taken from https://en.wikipedia.org/wiki/Enigma_rotor_details
        # These rotors represent the wiring of the rotors in the Enigma I machine.
        self.rotor_I   = [4, 10, 12, 5, 11, 6, 3, 16, 21, 25, 13, 19, 14, 22, 24, 7, 23, 20, 18, 15, 0, 8, 1, 17, 2, 9] # left
        self.rotor_II  = [0, 9, 3, 10, 18, 8, 17, 20, 23, 1, 11, 7, 22, 19, 12, 2, 16, 6, 25, 13, 15, 24, 5, 21, 14, 4] # middle
        self.rotor_III = [1, 3, 5, 7, 9, 11, 2, 15, 17, 19, 23, 21, 25, 13, 24, 4, 8, 22, 6, 0, 10, 12, 20, 18, 16, 14] # right
        self.rotors    = [self.rotor_I, self.rotor_II, self.rotor_III]
        # They are the index of the letter in the alphabet that is connected to the letter at the index.
        # When you pass a letter through the rotor, the letter is transformed into the letter at the index. We will see...

        # A reflector is a special rotor that is placed after the rotors. It is used to send the signal back through
        # the rotors in the opposite direction. The reflector does also rotate, but it does not have a turnover point.
        self.reflector = [24, 17, 20, 7, 16, 18, 11, 3, 15, 23, 13, 6, 14, 10, 12, 8, 4, 1, 5, 25, 2, 22, 21, 9, 0, 19]

        # Rotor positions and notch (turnover) points
        self.rotor_positions       = copy.deepcopy(rotor_positions)
        self.notch_positions       = [16, 4, 21]  # The rotors turn over when they reach these positions (Q, E, V in historical terms)

        # Plugboard setup (pairs of letter swaps)
        # The plugboard is a simple substitution cipher that swaps pairs of letters before the signal enters the rotors.
        # However, they are optional and can be omitted.
        self.plugboard = {}
        if plugboard_settings:
            self.plugboard = copy.deepcopy(plugboard_settings)

The plugboard_swap function returns the corresponding letter after the plugboard swap.

In [110]:
def plugboard_swap(self, letter):
    if letter in self.plugboard.keys():
        return self.plugboard[letter]
    return letter
    
Enigma.plugboard_swap = plugboard_swap


This function will be triggered when a keystroke occurs.


In [111]:
def rotate_rotors(self):
    # Rightmost rotor rotates every letter
    self.rotor_positions[0] = (self.rotor_positions[0] + 1) % 26
    # Middle rotor rotates when right rotor hits its notch
    if self.rotor_positions[0] == self.notch_positions[0]:
        self.rotor_positions[1] = (self.rotor_positions[1] + 1) % 26
    # Left rotor rotates when middle rotor hits its notch
    if self.rotor_positions[1] == self.notch_positions[1]:
        self.rotor_positions[2] = (self.rotor_positions[2] + 1) % 26
        
Enigma.rotate_rotors = rotate_rotors

The encrypt_letter function serves as a placeholder for the encryption process of the Enigma machine. This function takes a letter as input and returns the corresponding encrypted letter. The key is a number between 0 and 25, representing the shift of the letter within the alphabet.

Note: One of the Enigma machine's vulnerabilities was its limited number of possible keys, making it susceptible to brute force attacks.

In [112]:
def encrypt_letter(self, letter):
    # Apply plugboard swaps before going into the rotors
    letter = self.plugboard_swap(letter)
    # Convert letter to index (0-25 for A-Z)
    index = ord(letter) - ord('A')
    
    # Pass the letter through the rotors from right to left
    for i in range(len(self.rotors)):
        index = (self.rotors[i][(index + self.rotor_positions[i]) % 26] - self.rotor_positions[i]) % 26
    # Reflector transformation
    index = self.reflector[index]
    # Pass back through the rotors from left to right
    for i in range(len(self.rotors) - 1, -1, -1):
        index = (self.rotors[i].index((index + self.rotor_positions[i]) % 26) - self.rotor_positions[i]) % 26
    # Convert back to letter
    letter = chr(index + ord('A'))
    # Apply plugboard swaps after going through the rotors
    letter = self.plugboard_swap(letter)
    return letter
    
Enigma.encrypt_letter = encrypt_letter

In [113]:
def encrypt_message(self, message):
    encrypted_message = ""
    for letter in message:
        if letter.isalnum():
            self.rotate_rotors()
            encrypted_message += self.encrypt_letter(letter.upper())
        elif letter == " ":
            encrypted_message += " "
    return encrypted_message

Enigma.encrypt_message = encrypt_message

Let's finally have fun with the Enigma machine!

In [114]:
# Plugboard example: swapping A with Z, B with Y, etc.
plugboard_settings = {'A': 'Z', 'Z': 'A', 'B': 'Y', 'Y': 'B'}

# Set initial rotor positions
configuration      = [0, 0, 0]  # Starting at A-A-A
encrypter_enigma   = Enigma(configuration, plugboard_settings)

Play with it!

In [115]:
german_message    = "HELLO WETTER IS GOOD TODAY"
print("The message german wanted to send :", german_message)
encrypted_message = ""
for word in german_message:
    encrypted_message += encrypter_enigma.encrypt_message(word)
print("The message the codebreakers got  :", encrypted_message)


The message german wanted to send : HELLO WETTER IS GOOD TODAY
The message the codebreakers got  : MFNCA YHPMUB BE PFTL KZNOH


Here, we witness the birth of modern cryptography.

First of all, as not everyone knows, it was the Polish who first broke the Enigma code in the 1930s. They managed to decrypt it until the early 1940s when Germany changed the encryption methods and invaded Poland. Before the invasion, the Polish shared their knowledge of the Enigma with the British. The British then built the Bombe machine to break the new Enigma code.

The Bombe machine, often considered one of the earliest computers, was developed by Alan Turing, a British mathematician, logician, and computer scientist. Turing is regarded as the father of computer science and artificial intelligence. He played a crucial role in breaking the Enigma code during World War II, significantly aiding the Allied victory. Despite his heroic contributions, Turing was later prosecuted for..

But how was the Enigma code broken? Here are the key points:

The Enigma machine was originally a commercial product, used by many people. The Germans, however, used a more complex version of the machine. Nevertheless, many, including the Polish, knew how it worked and how it was built. So, contrary to popular belief, the Enigma machine was not a secret.

The Enigma machine was configured differently for each day, which made code-breaking more difficult. However, through espionage and other methods, the Allies managed to obtain the daily configurations. This marked the starting point of the code-breaking efforts.

Hitler accounted for many things, but he often overlooked the human factor. Enigma operators, for instance, would frequently reuse configurations and send the same messages over and over, without much caution. This lack of care played a significant role in helping the codebreakers.

A critical flaw of the Enigma machine was that it never encrypted a letter as itself. This became a crucial point in breaking the code.

The starting point for cracking the system was the word “Wetterbericht” (meaning “weather report” in German). The weather report was sent every day at the same time. This predictable pattern provided an essential clue for the codebreakers.

Finally, the codebreakers had two key pieces of information: the expected word “Wetter” and the knowledge that its encrypted version could not contain any repetition like "..XX.." (where "X" is any letter, and "." is any other letter). Using this, they began brute-forcing the system, focusing on finding the word “Wetter” within the encrypted messages.

In [116]:
#They tried to find the crypted version of "WETTER" to find the rotor positions
# Turing's big idea is to be able to try every possible rotor position and plugboard setting to find the correct one.
def Bombe(encrypted_message):
    words = encrypted_message.split(" ")
    for i in range(0,26):
        for j in range(0,26):
            for k in range(0,26):
                for m in range(0,len(words)):
                    strr = str(list(words[:m]) + ["WETTER"])
                    enigma = Enigma([i, j, k], plugboard_settings)
                    encrypted_word = enigma.encrypt_message(strr)
                    if encrypted_word.split(" ")[-1] in encrypted_message.split(" "):
                        enigma = Enigma([i, j, k], plugboard_settings)
                        desc = enigma.encrypt_message(encrypted_message)
                        return desc, [i, j, k]

                
    return None, None

decrypted_message, rotor_positions = Bombe(encrypted_message)
print("Rotor positions  :",  rotor_positions)
print("Decrypted message:", decrypted_message)


Rotor positions  : [0, 0, 0]
Decrypted message: HELLO WETTER IS GOOD TODAY
